### Imports

In [ ]:
%load_ext autoreload
%autoreload 2

#########################################################################################
#########################################################################################
####### BEGIN OPTIONAL SECTION
#########################################################################################
#########################################################################################
GLB_INSTALL_DEPENDENCIES = True

# Here to define dependencies
if GLB_INSTALL_DEPENDENCIES:
    !pip install transformers
    !pip install captum
    !pip install shap 
#########################################################################################
#########################################################################################
####### END OPTIONAL SECTION
#########################################################################################
#########################################################################################

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
import pandas as pd
import os
from os.path import join
import numpy as np

import torch

from transformers import BertForSequenceClassification, BertTokenizerFast#,  BertTokenizer
from transformers import AutoModelForSequenceClassification #AutoTokenizer
from transformers import BertTokenizer, BertConfig

import src.classification_model_utilities as mlclassif_utilities
import src.general_utilities as gral_utilities

from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

import scipy as sp
import shap

In [ ]:
model_id = "bert"

path_model = #"" Write path to the model

model_name = "bert-base-uncased"
#model_name = "GroNLP/hateBERT"

_num_classes = 2
_output_attentions = True
_output_hidden_states = True

## Read Dataset

In [ ]:
global_config_file = gral_utilities.read_config_file("config.yml")

PATH_DATASET = global_config_file["general_set_up"]["dataset_filename"]
PATH_DIR_INPUT = global_config_file["general_set_up"]["input_dir_name"]
PATH_DIR_DATASET = global_config_file["general_set_up"]["dataset_dir_name"]
INDEX_COLUMNS_DATASET = global_config_file["dataset"]["index_columns_dataset"] # 1
LIST_NAME_COLUMNS_DATASET = global_config_file["dataset"]["list_columns_names"]

df_dataset = mlclassif_utilities.import_dataset_from_excel(join(PATH_DIR_INPUT, PATH_DIR_DATASET, PATH_DATASET), INDEX_COLUMNS_DATASET, LIST_NAME_COLUMNS_DATASET)
df_dataset.head()

In [ ]:
list_sentences = df_dataset["text"]
list_sentences

## Load Model & tokenizer

In [ ]:
tokenizer = mlclassif_utilities.get_tokenizer(model_id, model_name) 
device = mlclassif_utilities.get_gpu_device_if_exists()

model = torch.load(path_model)
model.to(device)
model.eval()
model.zero_grad()

# Shap

In [ ]:
#tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=False)

def f(x):
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=512, truncation=True) for v in x]).cuda()
    outputs = model(tv)[0].detach().cpu().numpy()
    #print(outputs)
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores[:,1]) # use one vs rest logit units
    return val

In [ ]:
explainer = shap.Explainer(f, tokenizer)

In [ ]:
shap_values = explainer(list_sentences, fixed_context=1, batch_size=2)
#shap_values = explainer(list_sentences[0:50], fixed_context=1, batch_size=2)

In [ ]:
shap.plots.text(shap_values[:100], display=True)

In [ ]:
resultado_plots = shap.plots.text(shap_values[:100], display=False)

### Other visualizations

In [ ]:
shap.plots.bar(shap_values)

In [ ]:
shap.plots.waterfall(shap_values[0])

# Missclassifications

In [ ]:
import random

In [ ]:
ground_truth = df_dataset[df_dataset["role"]=="Witness"]["trauma"]
list_sentences = df_dataset[df_dataset["role"]=="Witness"]["text"]

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


all_spans_tokenized = mlclassif_utilities.get_all_spans_tokenized(
        tokenizer,
        list_sentences,
        _add_special_tokens = False, 
        _max_length = 512,
        _pad_to_max_length = True,
        _return_attention_mask = True, 
        type_tensors = "pt"
    )

input_ids = mlclassif_utilities.convert_list_into_pytorch_tensor(all_spans_tokenized[0])
attention_masks = mlclassif_utilities.convert_list_into_pytorch_tensor(all_spans_tokenized[1])
numeric_classes = mlclassif_utilities.convert_list_labels_into_pytorch_tensor(list(ground_truth))

input_ids = input_ids.to(device)
attention_masks = attention_masks.to(device)
numeric_classes = numeric_classes.to(device)

dataset = mlclassif_utilities.create_tensor_dataset(input_ids, attention_masks, numeric_classes)
dataloader = mlclassif_utilities.create_dataloader(dataset, 32)

def predict(model, device, dataloader):
    predictions = list()
    model.eval()

    for batch in dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():
            result = model(b_input_ids, 
                          token_type_ids=None, 
                          attention_mask=b_input_mask,
                          labels=b_labels,
                          return_dict=True)
        loss = result.loss
        logits = result.logits

        predictions.extend(logits.argmax(dim=1).cpu().numpy())
    return predictions

predictions = predict(model, device, dataloader)

In [ ]:
list_indices = list()
list_ground_truth = list()
list_predictions = list()
list_text_sentences = list()
list_sentences_2 = list(list_sentences)
for index, (true, prediction) in enumerate(zip(ground_truth, predictions)):
  if true != prediction:
    list_ground_truth.append(true)
    list_predictions.append(prediction)
    list_text_sentences.append(list_sentences_2[index])
    list_indices.append(index)

print(len(list_indices))

6132


In [ ]:
json_info = {"ground_truth": list_ground_truth, "predictions": list_predictions, "text": list_text_sentences}
df_info = pd.DataFrame.from_dict(json_info).reset_index()
#df_info.to_excel("Misclassifications_[writeSpecPattern].xlsx")